### Unit 41. 코루틴 사용하기  
메인 루틴에서 서브 루틴을 호출하면 서브 루틴의 코드를 실행한 뒤 다시 메인 루틴으로 돌아옴.  
서브 루틴이 끝나면 서브 루틴의 내용은 모두 사라짐.  
즉, 서브 루틴은 메인 루틴에 종속된 관계임.

In [ ]:
def add(a, b):
    c = a + b    # add 함수가 끝나면 변수와 계산식은 사라짐
    print(c)
    print('add 함수')
 
def calc():
    add(1, 2)    # add 함수가 끝나면 다시 calc 함수로 돌아옴
    print('calc 함수')
 
calc()

코루틴: 서로 협력하는 루틴이라는 뜻  
메인 루틴과 서브 루틴처럼 종속된 관계가 아니라 서로 대등한 관계이며 특정시점에 상대방의 코드를 실행

함수가 종료되지 되지 않은 상태에서 메인 루틴의 코드를 실행한 뒤 다시 돌아와서 코루틴의 코드를 실행. 따라서 코루틴이 종료되지 않았으므로 코루틴의 내용도 계속 유지.

일반 함수를 호출하면 코드를 한 번만 실행할 수 있지만, 코루틴은 코드를 여러 번 실행할 수 있음. 

__코루틴에 값 보내기__  
코루틴은 제너레이터의 특별한 형태.   
제너레이터는 yield로 값을 발생시키지만 코루틴은 yield로 값을 받아올 수 있음.   
코루틴에 값을 보내면서 값을 실행할 때는 send 메서드를 사용하고, 그 값을 받아오려면 (yield)형식으로 괄호로 묶어준 뒤 변수에 저장. 

In [1]:
def number_coroutine():
    while True:        # 코루틴을 계속 유지하기 위해 무한 루프 사용
        x = (yield)    # 코루틴 바깥에서 값을 받아옴, yield를 괄호로 묶어야 함
        print(x)
 
co = number_coroutine()
next(co)      # 코루틴 안의 yield까지 코드 실행(최초 실행)
 
co.send(1)    # 코루틴에 숫자 1을 보냄
co.send(2)    # 코루틴에 숫자 2을 보냄
co.send(3)    # 코루틴에 숫자 3을 보냄

1
2
3


__코루틴 바깥으로 값 전달하기__  
(yield 변수) 형식으로 yield에 변수를 지정한 뒤 괄호로 묶어주면 값을 받아오면서 바깥으로 값을 전달. 그리고 yield를 사용하여 바깥으로 전달한 값은 next 함수(__ next __ 메서드)와 send 메서드의 반환값으로 나오게 됨.

In [ ]:
def sum_coroutine():
    total = 0
    while True:
        x = (yield total)    # 코루틴 바깥에서 값을 받아오면서 바깥으로 값을 전달
        total += x
 
co = sum_coroutine()
print(next(co))      # 0: 코루틴 안의 yield까지 코드를 실행하고 코루틴에서 나온 값 출력
 
print(co.send(1))    # 1: 코루틴에 숫자 1을 보내고 코루틴에서 나온 값 출력
print(co.send(2))    # 3: 코루틴에 숫자 2를 보내고 코루틴에서 나온 값 출력
print(co.send(3))    # 6: 코루틴에 숫자 3을 보내고 코루틴에서 나온 값 출력

만약 코루틴을 강제로 종료하고 싶다면 close 메서드를 사용.

In [ ]:
def number_coroutine():
    while True:
        x = (yield)
        print(x, end=' ')
 
co = number_coroutine()
next(co)
 
for i in range(20):
    co.send(i)
 
co.close()    # 코루틴 종료

__GeneratorExit 예외 처리하기__  
코루틴 객체에서 close 메서드를 호출하면 코루틴이 종료될 때 GeneratorExit 예외가 발생. 따라서 이 예외를 처리하면 코루틴의 종료 시점을 알 수 있음.

In [2]:
def number_coroutine():
    try:
        while True:
            x = (yield)
            print(x, end=' ')
    except GeneratorExit:    # 코루틴이 종료 될 때 GeneratorExit 예외 발생
        print()
        print('코루틴 종료')
 
co = number_coroutine()
next(co)
 
for i in range(20):
    co.send(i)
 
co.close()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
코루틴 종료


__코루틴 안에서 예외 발생시키기__  
코루틴 안에 예외를 발생시켜서 코루틴을 종료해보자.  
코루틴 안에 예외를 발생 시킬 때는 throw 메서드를 사용.  
이때 throw 메서드에 지정한 에러 메시지는 except as의 변수에 들어감.


In [ ]:
def sum_coroutine():
    try:
        total = 0
        while True:
            x = (yield)
            total += x
    except RuntimeError as e:
        print(e)
        yield total    # 코루틴 바깥으로 값 전달
 
co = sum_coroutine()
next(co)
 
for i in range(20):
    co.send(i)
 
print(co.throw(RuntimeError, '예외로 코루틴 끝내기')) # 190
                                                      # 코루틴의 except에서 yield로 전달받은 값

__하위 코루틴의 반환값 가져오기__  
 yield from에 코루틴를 지정하면 해당 코루틴에서 return으로 반환한 값을 가져옴.  

 다음은 코루틴에서 숫자를 누적한 뒤 합계를 yield from으로 가져온다.



In [ ]:
def accumulate():
    total = 0
    while True:
        x = (yield)         # 코루틴 바깥에서 값을 받아옴
        if x is None:       # 받아온 값이 None이면
            return total    # 합계 total을 반환
        total += x
 
def sum_coroutine():
    while True:
        total = yield from accumulate()    # accumulate의 반환값을 가져옴
        print(total)
 
co = sum_coroutine()
next(co)
 
for i in range(1, 11):    # 1부터 10까지 반복
    co.send(i)            # 코루틴 accumulate에 숫자를 보냄
co.send(None)             # 코루틴 accumulate에 None을 보내서 숫자 누적을 끝냄
 
for i in range(1, 101):   # 1부터 100까지 반복
    co.send(i)            # 코루틴 accumulate에 숫자를 보냄
co.send(None)             # 코루틴 accumulate에 None을 보내서 숫자 누적을 끝냄

__연습문제: 문자열 검색 코루틴 만들기__  
다음 소스 코드를 완성하여 문자열에서 특정 단어가 있으면 True, 없으면 False가 출력되게 만드세요. find 함수는 코루틴으로 작성해야 합니다.


In [3]:
def find(word):
    result = False
    while True:
        line = (yield result)
        result = word in line

f = find('Python')
next(f)
 
print(f.send('Hello, Python!'))
print(f.send('Hello, world!'))
print(f.send('Python Script'))
 
f.close()

True
False
True


True

__심사문제: 사칙연산 코루틴 만들기__  
표준 입력으로 사칙연산 계산식이 여러 개 입력됩니다. 다음 소스 코드에서 각 계산식의 결과를 구하는 코루틴을 만드세요. 계산식은 문자열 형태이며 값과 연산자는 공백으로 구분됩니다. 그리고 값은 정수로 변환하여 사용하고, 나눗셈은 / 연산자를 사용하세요.

In [22]:
def calc():
    result = 0
    while True:
        cal = (yield result)
        a, op, b = cal.split()
        if op == '+':
            result = int(a) + int(b)
        elif op == '-':
            result = int(a) - int(b)
        elif op == '*':
            result = int(a) *  int(b)
        else:
            result = float(a) / float(b)


expressions = input().split(', ')
 
c = calc()
next(c)
 
for e in expressions:
    print(c.send(e))
 
c.close()

12
2.0
59


In [5]:
expressions

['1 + 2', '4 - 9']